In [15]:

import openai 
import os  

from dotenv import load_dotenv


In [16]:

load_dotenv(override=True)

client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_API_BASE")
)

print("API Key:", client.api_key[:10], "...")
print("Base URL:", client.base_url)


API Key: sk-or-v1-a ...
Base URL: https://openrouter.ai/api/v1/


In [17]:
response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[
        {"role": "user", "content": "Hello AI, please introduce yourself"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)


Hello! I’m an artificial intelligence created by OpenAI, designed to assist with a wide range of tasks. You can think of me as a conversational partner or a helpful tool—I can answer questions, provide explanations, generate creative content, assist with problem-solving, and much more. My knowledge is based on a vast amount of information up until October 2023, and I’m here to make your life easier or more interesting. Let me know how I can help you today! 😊


In [18]:
response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[{"role": "user", "content": "What is Python in one sentence?"}],
    max_tokens=500

)

ai_text = response.choices[0].message.content 


print("\nAI's Answer:")
print(ai_text)




AI's Answer:
Python is a high-level, interpreted, and general-purpose programming language known for its simplicity, readability, and versatility.


In [19]:
input_tokens = response.usage.prompt_tokens  
output_tokens = response.usage.completion_tokens  
total_tokens = response.usage.total_tokens    


print(f"  The question used: {input_tokens} tokens")
print(f"  AI's response used: {output_tokens} tokens")
print(f"  Total tokens billed: {total_tokens} tokens")




input_price_per_1k = 0.0008   
output_price_per_1k = 0.0032  


input_cost = (input_tokens / 1000) * input_price_per_1k
output_cost = (output_tokens / 1000) * output_price_per_1k
total_cost = input_cost + output_cost

print("\n Cost Breakdown for This Call:")
print(f"  Input cost:  ${input_cost:.6f} ({input_tokens} tokens)")
print(f"  Output cost: ${output_cost:.6f} ({output_tokens} tokens)")
print(f"  TOTAL COST:  ${total_cost:.6f}")



  The question used: 10 tokens
  AI's response used: 23 tokens
  Total tokens billed: 33 tokens

 Cost Breakdown for This Call:
  Input cost:  $0.000008 (10 tokens)
  Output cost: $0.000074 (23 tokens)
  TOTAL COST:  $0.000082


## LangChain

10+ lines of boilerplate. Want to switch to Google? Rewrite everything! brahhhhhh thanks to langChain

In [20]:

import os

def raw_openai_approach():
    
    import openai

    client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_API_BASE")
)

    response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[{"role": "user", "content": "Explain machine learning in one sentence"}],
    max_tokens=500

)

    if response:
        text = response.choices[0].message.content 
        print(f"Normal Client Calling : {text[:100]}...")
        return text

    return None

def langchain_approach():
    from langchain_openai import ChatOpenAI

    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",                    
        api_key=os.getenv("OPENROUTER_API_KEY"),      
        base_url=os.getenv("OPENROUTER_API_BASE"),      
        max_tokens=500, 
    )

    
    response = llm.invoke("Explain machine learning in one sentence")  

    if response:
        print(f"Langhain Response result : {response.content[:100]}...")
        return response.content

    return None

def main():
     
    raw_openai_approach()
    langchain_approach()

    
if __name__ == "__main__":
    main()

Normal Client Calling : **"Machine learning is a subset of artificial intelligence that enables systems to learn and improve...
Langhain Response result : "Machine learning is a subset of artificial intelligence that enables systems to automatically learn...


In [25]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def main():
    
    
    template = PromptTemplate(
        input_variables=["topic", "style"], 
        template="Explain {topic} in {style}" 
    )
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )

    if template and llm:
        
        test_prompt = template.format(
            topic="artificial intelligence", 
            style="exactly 5 words" 
        )

        print(f"Sending to AI: {test_prompt}")
        response = llm.invoke(test_prompt)
        print(f"\n AI Response: {response.content}")



if __name__ == "__main__":
    main()

Sending to AI: Explain artificial intelligence in exactly 5 words

 AI Response: Machines simulating human cognitive functions.


In [29]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

def main():
    

    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )   

    list_parser = CommaSeparatedListOutputParser() 

  
    list_prompt = PromptTemplate(
        template=  "List 3 benefits of {technology} (comma-separated):",
        input_variables=["technology"]
    )

    list_chain = list_prompt | llm | list_parser 
   
    if list_chain:
        result = list_chain.invoke({
            "technology": "cloud computing"
        })
        print(f" Input: 'List 3 benefits of cloud computing'")
        print(f"Parsed Output: {result}")
        print(f"Type: {type(result)}")
        print(f"Access items: result[0] = '{result[0] if result else ''}'")

    
    
    json_parser = JsonOutputParser()

    json_prompt = PromptTemplate(
        template="""Analyze {technology} and respond with JSON containing:
        - benefits: array of 2 benefits
        - complexity: low/medium/high
        - use_case: one main use case

        Technology: {technology}

        {format_instructions}""",
        input_variables=["technology"],
        partial_variables={"format_instructions": json_parser.get_format_instructions()}
    )

    json_chain = json_prompt | llm | json_parser 

   
    if json_chain:
        result = json_chain.invoke({
            "technology": "machine learning"
        })

        print(f"Input: 'Analyze machine learning'")

        try:
      
            parsed = result 

            print(f"   Benefits: {parsed.get('benefits', [])}")
            print(f"   Complexity: {parsed.get('complexity', 'N/A')}")
            print(f"   Use Case: {parsed.get('use_case', 'N/A')}")
            print(f"    Type: {type(parsed)}" )
        except (json.JSONDecodeError, TypeError, AttributeError):
            print(f" Parsing failed : {result}")



if __name__ == "__main__":
    main()

 Input: 'List 3 benefits of cloud computing'
Parsed Output: ['Here are three benefits of cloud computing:  ', '1. **Cost Efficiency** (reduces upfront infrastructure costs)', '', '2. **Scalability** (easily adjust resources based on demand)', '', '3. **Accessibility** (access data and applications from anywhere).  ', "Let me know if you'd like more details!"]
Type: <class 'list'>
Access items: result[0] = 'Here are three benefits of cloud computing:  '
Input: 'Analyze machine learning'
   Benefits: ['Automates complex data processing tasks', 'Enhances decision-making with predictive analytics']
   Complexity: high
   Use Case: Predictive maintenance in manufacturing
    Type: <class 'dict'>


## Complete Chain - Combining Everything!
## Build complete AI pipelines using LangChain's chain

In [30]:

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser

def main():
    

    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )  
    
    analysis_prompt = PromptTemplate(
        template="Analyze {technology} and provide pros and cons in 2-3 sentences",
        input_variables=["technology"]
    )

    str_parser = StrOutputParser()

    
    analysis_chain = analysis_prompt | llm | str_parser 

    
    if analysis_chain:
        result = analysis_chain.invoke({
            "technology": "blockchain"
        })
        print(f" Input: 'Analyze blockchain'")
        print(f" Output: {result}")


   
    list_prompt = PromptTemplate(
        template="List 3 use cases for {technology} (comma-separated):",
        input_variables=["technology"]
    )

    
    list_parser = CommaSeparatedListOutputParser()

    
    list_chain = list_prompt | llm | list_parser 

    if list_chain:
        result = list_chain.invoke({
            "technology": "blockchain"
        })
        print(f" Input: 'List use cases for blockchain'")
        print(f" Output: {result}")
        print(f" Type: {type(result)} - Python list!")

    
    test_tech = "artificial intelligence"
    print(f"Technology: {test_tech}\n")

    
    if analysis_chain and list_chain:
        analysis = analysis_chain.invoke({"technology": test_tech})
        print(f" Analysis:\n   {analysis}")

        use_cases = list_chain.invoke({"technology": test_tech})
        print(f"\n Use Cases:")
        for i, use_case in enumerate(use_cases, 1):
            print(f"   {i}. {use_case}")

if __name__ == "__main__":
    main()

 Input: 'Analyze blockchain'
 Output: Blockchain offers **decentralization**, **transparency**, and **security**, making it ideal for applications like cryptocurrency, supply chain tracking, and smart contracts. However, its **scalability** is limited, transaction speeds can be slow, and energy consumption (e.g., Proof of Work) raises environmental concerns. Additionally, regulatory uncertainty and potential for misuse (e.g., illicit activities) remain challenges.
 Input: 'List use cases for blockchain'
 Output: ['Here are three common use cases for blockchain (comma-separated):  ', '1. **Cryptocurrency transactions (e.g.', 'Bitcoin', 'Ethereum)**', '', '2. **Supply chain tracking and transparency**', '', '3. **Smart contracts for automated agreements**.  ', "Let me know if you'd like more details!"]
✅ Type: <class 'list'> - Python list!
Technology: artificial intelligence

 Analysis:
   ### **Pros of Artificial Intelligence (AI):**  
1. **Efficiency & Automation:** AI can process vast